# Scrape wiki page

In [1]:
import numpy as np
import pandas as pd
import urllib

Read the content from our url and store it into a variable

In [2]:
wikipage = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
content = wikipage.read()

Convert into string in utf-8 format

In [3]:
scontent = content.decode("UTF-8")

Extract the data from the table by looking for the < table > tags

In [4]:
wikitable = scontent[scontent.find("<table"):scontent.find("</table>")+8]

Now we are able to read the table with Pandas

In [5]:
mydata = pd.read_html(wikitable, header = 0)[0]

We have to process the cells that have an assigned borough and ignore the cells with a borough that is 'Not assigned'.

In [6]:
mydata = mydata[mydata.Borough != "Not assigned"]

If a cell has a borough but a 'Not assigned' neighborhood, then the neighborhood will be the same as the borough.

In [7]:
mydata.Neighbourhood[mydata.Neighbourhood == "Not assigned"] = mydata.Borough[mydata.Neighbourhood == "Not assigned"]

Let's see how our dataframe looks so far

In [8]:
mydata.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Group by Postcode and Borough and combine Neighbourhoods in one row seperated by comma

In [9]:
gdata = mydata.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda neighbour: ','.join(neighbour))
mydata = gdata.reset_index()

Below we can see that Neighbourhoods with the same postcode and borough are now combined in one row

In [10]:
mydata.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally we print the size of our dataframe

In [11]:
mydata.shape

(103, 3)

# Add geographical coordinates to my dataframe

In [14]:
# link to Geospatial data as csv on the web 
csv_url='https://cocl.us/Geospatial_data'
# pandas read csv from URL
geodata = pd.read_csv(csv_url)
geodata.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
mydata_geo = pd.merge(mydata,geodata, on=[mydata.Postcode, geodata["Postal Code"]])
mydata_geo.drop(['key_0','key_1','Postcode'], axis=1,inplace=True)
mydata_geo.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [18]:
mydata_geo = mydata_geo[["Postal Code", "Borough", "Neighbourhood", "Latitude", "Longitude"]]
mydata_geo.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
mydata_geo.head(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
